In [ ]:
import json
import binascii
import os
import urllib3
import certifi
import requests
from datetime import date, datetime, timedelta
import time

In [ ]:
#----------------------------------------------------------------------
# Чтение из файла search_url_list.txt поисковых запросов по сайту
# ----------------------------------------------------------------------
def get_search_url(search_url_file = 'search_url_list.txt'):

    search_url_list = []

    with open(search_url_file, 'r') as f:
        urls = f.readlines()

    for current_string in urls:
        if current_string[0:1] == '#' or current_string[0:1] == '\n':
            continue
        search_url_list.append(current_string.rstrip())

    return search_url_list


In [126]:
def get_vacancies(url_list, vacancies, vacancy_with_response, log_file_name='vacancy_log.txt'):

    date_from = (date.today() - timedelta(hours=24)).isoformat()
    searches = []
    new_vacancies = []

    with open(log_file_name, 'a') as logfile:

        for url in url_list:
            for page in range(50):
                try:
                    # searches.append(requests.get(url + '&page={}&per_page=100&date_from={}'.format(page, date_from)).json())
                    searches.append(requests.get(url).json())
                except Exception as ex_err:
                    logfile.write('==> ERROR: {} W= Read Vacansies ===> {}'.format(date.today().isoformat(), ex_err) + '\n')

        try:
            for items in searches:
                for j in items['items']:
                    if j['id']  in vacancies:
                        continue
                    elif j['id'] in vacancy_with_response:
                        continue
                    else:
                        vacancies.append(j['id'])
                        new_vacancies.append(j['id'])
        except:
            pass

    return new_vacancies

In [143]:
#-----------------------------------------------------------------
# Получает список вакансий, на которые ранее были сделаны отклики,
# в глобальную переменную vacancy_with_response = []
#-----------------------------------------------------------------
def get_vacancy_with_response(pages = 500):

    def http_request(page_number = 0):
        return http.request('GET', 'https://api.hh.ru/negotiations?&page={}&per_page=100'.format(page_number),
                                        headers = {'Authorization' : 'Bearer {}'.format(user_token_params['access_token']),
                                                   'Accept'        : '*/*',
                                                   'User-Agent'    : 'CIO_jbSearch (ssv.ruby@gmail.com)'})


    http = urllib3.PoolManager(cert_reqs='CERT_REQUIRED', ca_certs=certifi.where())

    for page_num in range(pages):
        req = http_request(page_num)
        data = json.loads(req.data)
        if len(data['items']) == 0:
            break
        for item in data['items']:
            vacancy_with_response.append(item['vacancy']['id'])


In [ ]:
# GET /resumes/{resume_id}/similar_vacancies

In [ ]:
# respond_to_vacancies
#---------------------------
def respond_to_vacancies(vacancies, resume_id, message='', log_file_name='vacancy_log.txt'):

    http = urllib3.PoolManager(cert_reqs='CERT_REQUIRED', ca_certs=certifi.where())

    with open(log_file_name, 'a') as logfile:

        for vacancy_id in vacancies:
            try:

                body, content_type = encode_multipart_formdata({'vacancy_id': '{}'.format(vacancy_id),
                                     'resume_id' : '{}'.format(resume_id),
                                     'message'   : '{}'.format(message)})

                r = http.request('POST', 'https://api.hh.ru/negotiations',
                                        headers = {'Authorization' : 'Bearer {}'.format(user_token_params['access_token']),
                                                  'Accept'        : '*/*',
                                                  'User-Agent'    : 'CIO_jbSearch (ssv.ruby@gmail.com)',
                                                  'Content-Type'  : '{}'.format(content_type)},
                                        body =  '{}'.format(body)
                                      )

                result = 'Vacancy_id: {}, Date: {}, Status: {}, Reason: {}, Response_link: {}'.format(vacancy_id, r.getheader('Date'),
                                                                                r.status, r.reason,
                                                                                'https://hh.ru/applicant{}'.format(r.getheader('Location')))
                logfile.write('{} == Respond Vacansies ===> {}'.format(date.today().isoformat(), result) + '\n')

            except Exception as ex_err:
                logfile.write('{} W= Respond ERROR ===> {}'.format(date.today().isoformat(), ex_err) + '\n')

In [144]:
# ----------------------------------------------------------------------
#  Список просмотренных программой вакансий (Глобальная переменная): vacancies = []
# ----------------------------------------------------------------------
vacancies = []                                       #  Список просмотренных вакансий в текущем сеансе (Глобальная переменная)
vacancy_with_response = []                           #  Список просмотренных вакансий за всю историю (Глобальная переменная)
resume_id = '6e1019f2ff077703530039ed1f4b59356a4b71' # IT Directot
period    = 900                                      # 900 сек - период обращения к сайту

In [ ]:
# Get authorisation params
with open('user_token_params.json', 'r', encoding='utf-8') as f:
    user_token_params = json.load(f)

In [141]:
url_list = []
url_list.append('https://api.hh.ru/vacancies?&text=%D0%BF%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82')
#url_list.append('https://api.hh.ru/vacancies?text=%D0%9F%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82+data+science')

In [145]:
get_vacancy_with_response()

In [146]:
len(vacancy_with_response)

667

In [147]:
get_vacancy_with_response()
len(vacancy_with_response)
new_vacancies = get_vacancies(url_list, vacancies, vacancy_with_response)
len(vacancies)

18